<a href="https://colab.research.google.com/github/koad7/NLP_PYTORCH/blob/main/Unsupervised_NER_With_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [6]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to convert sentences to embeddings
def sentence_to_embedding(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt')

    # Get the embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # The embeddings of the [CLS] token can be used as the embedding of the sentence
    sentence_embedding = outputs.last_hidden_state[0, 0, :].numpy()

    return sentence_embedding

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from sklearn.cluster import KMeans
import numpy as np




In [74]:
sentence = '''When Jack Ma was allocating company shares, he gave Tong 0.2% shares right saying it will worth 100 million when Alibaba goes public into the market later. He asked her to stay in the company, not change to other companies, and she will get 100 million when Alibaba goes public.
She waited years but Alibaba never went public. She asked Jack when will Alibaba go public in 2004 and Jack said soon. She asked again in 2006 and Jack Ma said soon, again.
However, she never saw it happen and never got 100 million. When Alibaba finally went public on the New York Stock Exchange in September 2014, it was worth 245.7 billion. Tong, the former senior receptionist, and current Vice President of Alibaba, became a millionaire from a receptionist with 320 million worth.
Entrepreneurship is a 16-character word where the 6th letter P stands for Patience. Most startups end in this 6th stage because of a lack of patience.
Whenever I am going to give up my dreams I read this story when a receptionist has this much patience then how can I give up, I am born to be an Entrepreneur.'''

def count_words(text):
    words = text.split()
    return len(words)
print(f'We have {count_words(sentence)} total number of words.')
# Tokenize the sentence and get the corresponding input IDs
inputs = tokenizer(sentence, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    outputs = model(**inputs)

# The embeddings are stored in the last hidden state
embeddings = outputs.last_hidden_state

We have 193 total number of words.


In [75]:
embeddings.size()

torch.Size([1, 237, 768])

In [76]:
import numpy as np

# Transofmr tensor from 3D to 2D
embeddings_2d = embeddings.view(embeddings.size(1), -1)
# Initialize a KMeans object with 2 clusters (for 2 classes: NE and non-NE)
kmeans = KMeans(n_clusters=2, random_state=0)


In [77]:
# Now fit the KMeans model
labels = kmeans.fit_predict(embeddings_2d)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [78]:
print(f'Total siez: {labels.size} \n0: {np.count_nonzero(labels)}\n1: {labels.size-np.count_nonzero(labels)}')


Total siez: 237 
0: 176
1: 61


In [79]:
count_0 = labels.size-np.count_nonzero(labels)
count_1 = np.count_nonzero(labels)
if  count_1 > count_0:
  labels = np.logical_not(labels).astype(int)

In [92]:
tokens = tokenizer.tokenize(sentence)

In [93]:
tokens.append("")

In [94]:
tokens.append("")

In [83]:
import pandas as pd

In [95]:
pd.DataFrame({'tokens':tokens, 'labels':labels})

,tokens,labels
0,when,0
1,jack,0
2,ma,0
3,was,0
4,all,0
...,...,...
232,an,0
233,entrepreneur,0
234,.,1
235,,0


In [1]:
import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")
sentence = ('It has also arisen in criminal justice, healthcare, and hiring, compounding existing racial, economic, and gender biases.')
doc = nlp(sentence)
tokens = [token.text for token in doc]
tokens


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


['It',
 'has',
 'also',
 'arisen',
 'in',
 'criminal',
 'justice',
 ',',
 'healthcare',
 ',',
 'and',
 'hiring',
 ',',
 'compounding',
 'existing',
 'racial',
 ',',
 'economic',
 ',',
 'and',
 'gender',
 'biases',
 '.']

In [5]:
from collections import Counter
import pandas as pd
bag_of_words = Counter(tokens)
bag_of_words

Counter({'It': 1,
         'has': 1,
         'also': 1,
         'arisen': 1,
         'in': 1,
         'criminal': 1,
         'justice': 1,
         ',': 5,
         'healthcare': 1,
         'and': 2,
         'hiring': 1,
         'compounding': 1,
         'existing': 1,
         'racial': 1,
         'economic': 1,
         'gender': 1,
         'biases': 1,
         '.': 1})

In [6]:
bag_of_words.most_common(3)

[(',', 5), ('and', 2), ('It', 1)]

In [7]:
counts = pd.Series(dict(bag_of_words.most_common()))
counts

,              5
and            2
It             1
has            1
also           1
arisen         1
in             1
criminal       1
justice        1
healthcare     1
hiring         1
compounding    1
existing       1
racial         1
economic       1
gender         1
biases         1
.              1
dtype: int64

In [8]:
counts / counts.sum()

,              0.217391
and            0.086957
It             0.043478
has            0.043478
also           0.043478
arisen         0.043478
in             0.043478
criminal       0.043478
justice        0.043478
healthcare     0.043478
hiring         0.043478
compounding    0.043478
existing       0.043478
racial         0.043478
economic       0.043478
gender         0.043478
biases         0.043478
.              0.043478
dtype: float64

In [10]:
sentences2 = '''
Algorithmic bias describes systematic and repeatable errors in a computer system that create
unfair outcomes, such as privileging one arbitrary group of users over others. Bias can emerge
due to many factors, including but not limited to the design of the algorithm or the unintended or
unanticipated use or decisions relating to the way data is coded, collected, selected or used to
train the algorithm. Algorithmic bias is found across platforms, including but not limited to
search engine results and social media platforms, and can have impacts ranging from
inadvertent privacy violations to reinforcing social biases of race, gender, sexuality, and
ethnicity. The study of algorithmic bias is most concerned with algorithms that reflect
"systematic and unfair" discrimination. This bias has only recently been addressed in legal
frameworks, such as the 2018 European Union’s General Data Protection Regulation. More
comprehensive regulation is needed as emerging technologies become increasingly advanced
and opaque.
As algorithms expand their ability to organize society, politics, institutions, and behavior,
sociologists have become concerned with the ways in which unanticipated output and
manipulation of data can impact the physical world. Because algorithms are often considered to
be neutral and unbiased, they can inaccurately project greater authority than human expertise,
and in some cases, reliance on algorithms can displace human responsibility for their outcomes.
Bias can enter into algorithmic systems as a result of pre-existing cultural, social, or
institutional expectations; because of technical limitations of their design; or by being used in
unanticipated contexts or by audiences who are not considered in the software’s initial design.
Algorithmic bias has been cited in cases ranging from election outcomes to the spread of online
hate speech. It has also arisen in criminal justice, healthcare, and hiring, compounding existing
racial, economic, and gender biases. The relative inability of facial recognition technology to
accurately identify darker-skinned faces has been linked to multiple wrongful arrests of men of
color, an issue stemming from imbalanced datasets. Problems in understanding, researching,
and discovering algorithmic bias persist due to the proprietary nature of algorithms, which are
typically treated as trade secrets. Even when full transparency is provided, the complexity of
certain algorithms poses a barrier to understanding their functioning. Furthermore, algorithms
may change, or respond to input or output in ways that cannot be anticipated or easily
reproduced for analysis. In many cases, even within a single website or application, there is no
single "algorithm" to examine, but a network of many interrelated programs and data inputs,
even between users of the same service.
'''

In [11]:
tokens = [tok.text for tok in nlp(sentences2)]

In [13]:
counts = Counter(tokens)

In [15]:
import requests

In [16]:
url = ('https://gitlab.com/tangibleai/nlpia2/-/raw/main/src/nlpia2/ch03/bias_intro.txt')
response = requests.get(url)

In [17]:
bias_intro = response.content.decode()
bias_intro[:60]

'Algorithmic bias describes systematic and repeatable errors '

In [18]:
tokens = [tok.text for tok in nlp(bias_intro)]
counts = Counter(tokens)
counts.most_common(5)

[(',', 35), ('of', 16), ('.', 16), ('to', 15), ('and', 14)]

In [21]:
docs = [nlp(s) for s in bias_intro.split('\n') if s.strip()]

In [24]:
counts = []
for doc in docs:
  counts.append(Counter([t.text.lower() for t in doc]))

In [26]:
df = pd.DataFrame(counts)

In [28]:
df = df.fillna(0).astype(int)

In [29]:
df.head()

,algorithmic,bias,describes,systematic,and,repeatable,errors,in,a,computer,...,there,no,examine,network,interrelated,programs,inputs,between,same,service
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
docs = list(nlp(bias_intro).sents)

In [32]:
counts = []
for doc in docs:
  counts.append(Counter([t.text.lower() for t in doc]))

In [33]:
df = pd.DataFrame(counts)
df = df.fillna(0).astype(int)

In [35]:
docs_tokens = []
for doc_text in docs:
  doc_text = doc_text.text.lower()
  docs_tokens.append([tok.text for tok in nlp(doc_text)])


In [44]:
all_doc_tokens = []
for doc_tokens in docs_tokens:
  all_doc_tokens.extend(doc_tokens)


In [46]:
vocab = sorted(set(all_doc_tokens))


In [59]:
lexicon = vocab[9:]

In [60]:
from collections import OrderedDict
zero_vector = OrderedDict((token, 0) for token in lexicon)


In [61]:
zero_vector

OrderedDict([('a', 0),
             ('ability', 0),
             ('accurately', 0),
             ('across', 0),
             ('addressed', 0),
             ('advanced', 0),
             ('algorithm', 0),
             ('algorithmic', 0),
             ('algorithms', 0),
             ('also', 0),
             ('an', 0),
             ('analysis', 0),
             ('and', 0),
             ('anticipated', 0),
             ('application', 0),
             ('arbitrary', 0),
             ('are', 0),
             ('arisen', 0),
             ('arrests', 0),
             ('as', 0),
             ('audiences', 0),
             ('authority', 0),
             ('barrier', 0),
             ('be', 0),
             ('because', 0),
             ('become', 0),
             ('been', 0),
             ('behavior', 0),
             ('being', 0),
             ('between', 0),
             ('bias', 0),
             ('biases', 0),
             ('but', 0),
             ('by', 0),
             ('can', 0),
           

In [63]:
import copy
doc_vectors = []
for doc in docs:
  vec = copy.copy(zero_vector)
  tokens = [token.text for token in nlp(doc.text.lower())]
  token_counts = Counter(tokens)
  for key, value in token_counts.items():
    vec[key] = value / len(lexicon)
  doc_vectors.append(vec)

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = docs
vectorizer = CountVectorizer()


In [74]:
corpus = [doc for doc in corpus]
count_vectors = vectorizer.fit_transform(corpus)

In [75]:
print(count_vectors.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


In [77]:
import numpy as np
v1 = np.array(list(range(5)))
v2 = pd.Series(reversed(range(5)))
slow_answer = sum([4.2 * (x1 * x2) for x1, x2 in zip(v1, v2)])
slow_answer

42.0

In [79]:
import math
def cosine_sim(vec1, vec2):
  vec1 = [val for val in vec1.values()]
  vec2 = [val for val in vec2.values()]

  dot_prod = 0
  for i, v in enumerate(vec1):
    dot_prod += v * vec2[i]

  mag_1 = math.sqrt(sum([x**2 for x in vec1]))
  mag_2 = math.sqrt(sum([x**2 for x in vec2]))

  return dot_prod / (mag_1 * mag_2)

In [80]:
from this import s

print(s)

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
Gur Mra bs Clguba, ol Gvz Crgref

Ornhgvshy vf orggre guna htyl.
Rkcyvpvg vf orggre guna vzcyvpvg.
Fvzcyr vf orggre guna pbzcyrk.
Pbzcyrk vf or

In [81]:
import this as a


In [86]:
DATA_DIR = ('https://gitlab.com/tangibleai/nlpia/-/raw/master/src/nlpia/data')
url = DATA_DIR + '/machine_learning_full_article.txt'
ml_text = requests.get(url).content.decode()
ml_char_frequencies = char_vectorizer.fit_transform(ml_text)
generate_histogram(s_char_frequencies, s_char_vectorizer)

ValueError: ignored